In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore',category = DeprecationWarning)

In [ ]:
df = pd.read_csv('/content/order_item_feedback_original.csv')


In [ ]:
df.head()

,feedback_id,order_name,customer_id,product_id,variant_id,overall_rating,review_title,review_details,display_on_website,is_spam,created_at,updated_at,customer_name,customer_mobile,is_active
0,163749,NH-1982745,NaN,6721696137304,40072814919768,5,Very effective,I never thought that something could work on t...,False,False,2024-05-08 23:52:14.469141 +05:30,2024-05-08 23:53:57.149285 +05:30,NaN,NaN,False
1,201,NH-267658,NaN,4360782807128,33487531573336,5,In love with it!,Malai in real sense. Blends with your skin to ...,False,False,2021-12-11 15:38:58.564182 +05:30,2021-12-11 15:40:38.781939 +05:30,Anonymous,NaN,True
2,163760,NH-2026445,NaN,8474503020850,45648648896818,5,Amazing Packs,"Just Wow, after using it for almost 15days, i'...",False,False,2024-05-09 19:33:54.873823 +05:30,2024-05-09 19:34:58.678912 +05:30,NaN,NaN,False
3,164570,NH-1616252,NaN,4360782807128,40125053337688,5,Heavenly smell !,It smells amazing and is very moisturising. Gr...,False,False,2024-06-15 22:59:53.581367 +05:30,2024-06-15 23:00:44.478209 +05:30,NaN,NaN,False
4,164568,NH-1616252,NaN,6586009649240,39358487494744,5,Spa like feel!,This is such a lovely face pack and quite vers...,False,False,2024-06-15 22:49:21.630177 +05:30,2024-06-15 22:52:55.716308 +05:30,NaN,NaN,False


In [ ]:
# Concatenate the 'review_title' and 'review_details' columns
df['combined_review'] = df['review_title'] + ' ' + df['review_details']

In [ ]:
review_df= df[['combined_review']]

In [ ]:
review_df['combined_review'].isna().sum()

1

In [ ]:
review_without_nan = review_df.copy()
review_without_nan.dropna(inplace=True)

## **Text preprocessing**

###Lower casing

In [ ]:
#lower casing the text
review_without_nan['combined_review'] = review_without_nan['combined_review'].str.lower()

### Remove html tags

In [ ]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [ ]:
# #funtion to remove html tags gerenerally we use
# import re
# def remove_html_tags(text):
#     pattern = re.compile('<.*?>')
#     return re.sub(pattern, '', text)

In [ ]:
#removing the html tags
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(remove_html_tags)

<ipython-input-9-273477798574>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


###URL Handling

In [ ]:
import re

In [ ]:
def remove_urls(text):
  pattern= re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'', text)

In [ ]:
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(remove_urls)

### Punctuation Removal

In [ ]:
import string
import re
import time
import pandas as pd

def improved_text_cleaning(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove 'n' when it's not part of a word
    text = re.sub(r'\s+n\s+', ' ', text)

    # Remove extra spaces
    text = ' '.join(text.split())

    return text

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Punctuation removal apply
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(improved_text_cleaning)

### Chat word removal

In [ ]:
chat_words_dict = {
    "brb": "Be Right Back",
    "lol": "Laugh Out Loud",
    "rofl": "Rolling On the Floor Laughing",
    "omg": "Oh My God",
    "ttyl": "Talk To You Later",
    "idk": "I Don’t Know",
    "smh": "Shaking My Head",
    "imo": "In My Opinion",
    "imho": "In My Humble Opinion",
    "fyi": "For Your Information",
    "btw": "By The Way",
    "nvm": "Never Mind",
    "tbh": "To Be Honest",
    "gtg": "Got To Go",
    "bff": "Best Friends Forever",
    "jk": "Just Kidding",
    "ttfn": "Ta-Ta For Now",
    "yolo": "You Only Live Once",
    "afk": "Away From Keyboard",
    "irl": "In Real Life",
    "dm": "Direct Message",
    "pm": "Private Message",
    "gg": "Good Game",
    "ftw": "For The Win",
    "ppl": "People",
    "thx": "Thanks",
    "lmao": "Laughing My Ass Off",
    "icymi": "In Case You Missed It",
    "fomo": "Fear Of Missing Out",
    "omw": "On My Way",
    "idc": "I Don’t Care",
    "asap": "As Soon As Possible",
    "b/c": "Because",
    "cul8r": "See You Later",
    "gr8": "Great",
    "k": "Okay",
    "l8r": "Later",
    "np": "No Problem",
    "tmi": "Too Much Information",
    "ty": "Thank You",
    "yw": "You’re Welcome",
    "wtf": "What The Fuck",
    'u': 'you',
    'ur': 'your',
    'r': 'are',
    'lol': 'laughing out loud',
    'omg': 'oh my god',
    'idk': 'I do not know',
    'btw': 'by the way',
    'brb': 'be right back',
    'b4': 'before',
    'gr8': 'great',
    'bday': 'birthday',
    'plz': 'please',
    'thx': 'thanks',
    'np': 'no problem',
    'cuz': 'because',
    'msg': 'message',
    'wanna': 'want to',
    'gonna': 'going to'
}


In [ ]:
import re

def replace_chat_words(text, chat_words_dict):
    chat_words_re = re.compile(r'\b(' + '|'.join(chat_words_dict.keys()) + r')\b')
    return chat_words_re.sub(lambda x: chat_words_dict[x.group()], text)

In [ ]:
# Define the function to preprocess the text
def preprocess_text(text):
    # Replace chat words
    text = replace_chat_words(text, chat_words_dict)
    return text

In [ ]:
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(preprocess_text)

### Emoji handler

In [ ]:
!pip install emoji
import emoji
# Function to replace emojis with text descriptions
def replace_emojis(text):
    return emoji.demojize(text)
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(replace_emojis)

###Stop word Removal

In [ ]:
pip install nltk langdetect indic-nlp-library

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving stop_hinglish.txt to stop_hinglish.txt
Saving stop_hindi.txt to stop_hindi.txt


In [ ]:
# Assuming the uploaded file is 'file.txt'
with open('stop_hinglish.txt', 'r') as file:
    stop_hinglish = file.read()

In [ ]:
# Assuming the uploaded file is 'file.txt'
with open('stop_hindi.txt', 'r') as file:
    stop_hindi = file.read()

In [ ]:
type(stop_hinglish)

str

In [ ]:
type(stop_hindi)

str

In [ ]:
hinglish_stopwords = []
for i in stop_hinglish.split():
  hinglish_stopwords.append(i)

In [ ]:
hindi_stopwords = []
for i in stop_hindi.split():
  hindi_stopwords.append(i)

In [ ]:
import re
from langdetect import detect
from nltk.corpus import stopwords
from indicnlp.tokenize import indic_tokenize

# English stopwords (from NLTK)
english_stopwords = set(stopwords.words('english'))

# Hinglish stopwords (custom list)
hinglish_stopwords = set(hinglish_stopwords)

# Hindi stopwords (custom list or from a Hindi stopword corpus)
hindi_stopwords = set(hindi_stopwords)

# Function to detect language and remove corresponding stopwords
def remove_stopwords_multilingual(review):
    try:
        # Detect the language of the review
        lang = detect(review)

        # Tokenize the review into words
        words = review.split()

        if lang == 'en':  # English
            return ' '.join([word for word in words if word.lower() not in english_stopwords])
        elif lang == 'hi':  # Hindi
            # Tokenize Hindi text with indicnlp
            words = indic_tokenize.trivial_tokenize(review)
            return ' '.join([word for word in words if word not in hindi_stopwords])
        else:  # Assume Hinglish for unknown language or if Hinglish detected
            return ' '.join([word for word in words if word.lower() not in hinglish_stopwords])

    except Exception as e:
        print(f"Error processing review: {review}, Error: {str(e)}")
        return review  # Return original text if error occurs

In [ ]:
#stopword removal
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(remove_stopwords_multilingual)

### Final processing

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(lambda x:' '.join([word for word in word_tokenize(x) if word not in [english_stopwords,hinglish_stopwords,hindi_stopwords] and len(word)> 2]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
review_without_nan

### Spell checker

In [ ]:
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from spellchecker import SpellChecker

# Initialize the spell checker
spell = SpellChecker()

# Function to correct spelling in a sentence
def correct_spelling(text):
    if pd.isna(text) or not isinstance(text, str):
        return text  # Return the original value if it's NaN or not a string

    # Tokenize the text into words
    words = text.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)
    # Correct the misspelled words and ensure all are strings
    corrected_words = [spell.correction(word) if word in misspelled_words else word for word in words]
    # Filter out None values and ensure all are strings
    corrected_words = [word if isinstance(word, str) else '' for word in corrected_words]
    # Join corrected words into a sentence
    return ' '.join(corrected_words)


In [ ]:
# Apply the correction function to the 'combined_review' column
review_without_nan['combined_review_corrected'] = review_without_nan['combined_review'].apply(correct_spelling)

In [ ]:
# Show the updated DataFrame
review_without_nan[['combined_review', 'combined_review_corrected']].head()

In [ ]:
# Export to a CSV file
review_without_nan.to_csv('final_df_export.csv', index=False)

###Topic modelling

In [ ]:
import pandas as pd
import numpy as np
review_without_nan = pd.read_csv('/content/final_df_export.csv')

In [ ]:
review_without_nan.isna().sum()

,0
combined_review,8
combined_review_corrected,9


In [ ]:
review_without_nan

In [ ]:
review_for_tm = review_without_nan[['combined_review_corrected']].copy()
review_for_tm= review_for_tm.dropna()

In [ ]:
review_for_tm.head()

In [ ]:
review_for_tm.isna().sum()

,0
combined_review_corrected,0


In [ ]:
review_for_tm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1050 entries, 0 to 1058
Data columns (total 1 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   combined_review_corrected  1050 non-null   object
dtypes: object(1)
memory usage: 16.4+ KB


In [ ]:
!pip install bertopic

In [ ]:
!pip install sentence-transformers
!pip install umap-learn hdbscan

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


In [ ]:
reviews = review_for_tm['combined_review_corrected'].tolist()

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model.encode(reviews, show_progress_bar=True)

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
topic_model = BERTopic(verbose=True)

In [ ]:
topics, probabilities = topic_model.fit_transform(reviews, embeddings)

2024-09-10 08:12:16,874 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-10 08:12:33,593 - BERTopic - Dimensionality - Completed ✓
2024-09-10 08:12:33,596 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-10 08:12:33,641 - BERTopic - Cluster - Completed ✓
2024-09-10 08:12:33,648 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-10 08:12:33,712 - BERTopic - Representation - Completed ✓


In [ ]:
topics_overview = topic_model.get_topic_info()
print(topics_overview)

In [ ]:
for topic in range(len(topics_overview)):
    print(f"Topic {topic}: {topic_model.get_topic(topic)}")


Topic 0: [('hair', 0.08931692924965699), ('oil', 0.04613977244929543), ('mask', 0.03737394286608778), ('hairball', 0.033250559874477344), ('shampoo', 0.030369479746848282), ('fall', 0.02857292118486759), ('scalp', 0.02728770608649708), ('wash', 0.024750817543829677), ('using', 0.023561492425270104), ('product', 0.021950056286942243)]
Topic 1: [('skin', 0.07948312276726661), ('face', 0.04903795822932793), ('moisturizer', 0.029368297030264775), ('product', 0.027639801525995975), ('using', 0.026015871652735542), ('dry', 0.02324221808134951), ('acne', 0.022144825638750247), ('glow', 0.022144825638750247), ('good', 0.021125374432069813), ('feel', 0.019989890853025903)]
Topic 2: [('henna', 0.10944891687013825), ('color', 0.07500025411364149), ('paste', 0.06655567399076619), ('hair', 0.04956977059827246), ('hyena', 0.04593687028751376), ('brown', 0.037895025253999015), ('ready', 0.030365596503021574), ('use', 0.0290775511336778), ('product', 0.028418187827492187), ('good', 0.02578336643824662

In [ ]:
# Visualize topic clusters
topic_model.visualize_topics()

# Visualize the distribution of topics
topic_model.visualize_distribution(probabilities)

In [ ]:
import pickle

# Save the embeddings and topic model
with open('bertopic_model.pkl', 'wb') as f:
    pickle.dump((topic_model, embeddings), f)